In [1]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import geocoder # import geocoder
import requests 
from bs4 import BeautifulSoup 

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
import sys
print('Libraries imported.')

Libraries imported.


# Segmenting and Clustering Neighborhoods in Toronto

In [2]:

URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
r = requests.get(URL) 
  
soup = BeautifulSoup(r.content, 'html5lib') 
table = soup.find('div', attrs = {'id':'container'}) 

# print(soup.prettify()) 
print('Page Scrapped.')

Page Scrapped.


In [3]:
postalCodes = [];
boroughs= [];
neighborhoods = [];
columnNum = 1;
passVal = False

for row in soup.find_all('td'):
    for cell in row:
        if cell.string and cell.string[0].isalpha() and len(cell.string) > 2:
            passVal = False
            if columnNum == 1:
                if passVal == False and cell.string[1].isdigit():
                    postalCodes.append(cell.string);   
                    columnNum = 2
                else:
                    continue
            elif columnNum == 2 :
                if cell.string == 'Not assigned':
                    passVal = True
                    del postalCodes[-1]
                    columnNum = 1
                    continue
                else:
                    boroughs.append(cell.string);      
                    columnNum = 3
            elif columnNum == 3 :
                if cell.string == 'Not assigned\n':
                    neighborhoods.append(boroughs[-1])
                else:
                    neighborhoods.append(cell.string); 
                columnNum = 1
                
print('Data Collected.')

Data Collected.


In [4]:
# define the dataframe columns
column_names = ['PostalCode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighbors = pd.DataFrame(columns=column_names)

neighbors

,PostalCode,Borough,Neighborhood,Latitude,Longitude


In [5]:
lat_lng_coords = None

for data in range(0, len(postalCodes)-1):
    code = postalCodes[data]
    borough = boroughs[data]
    neighborhood_name = neighborhoods[data]
    
    g = geocoder.arcgis('{}, Toronto, Ontario'.format(code))
    lat_lng_coords = g.latlng

    neighbors = neighbors.append({ 'PostalCode': code,
                                   'Borough': borough,
                                   'Neighborhood': neighborhood_name,
                                   'Latitude': lat_lng_coords[0],
                                   'Longitude': lat_lng_coords[1]}, ignore_index=True)
    
neighbors

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1A\n,Not assigned\n,Not assigned\n,43.64869,-79.38544
1,M2A\n,Not assigned\n,Not assigned\n,43.64869,-79.38544
2,M3A\n,North York\n,Parkwoods\n,43.75188,-79.33036
3,M4A\n,North York\n,Victoria Village\n,43.73042,-79.31282
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n",43.65514,-79.36265
...,...,...,...,...,...
174,M4Z\n,Not assigned\n,Not assigned\n,43.64869,-79.38544
175,M5Z\n,Not assigned\n,Not assigned\n,43.64869,-79.38544
176,M6Z\n,Not assigned\n,Not assigned\n,43.64869,-79.38544
177,M7Z\n,Not assigned\n,Not assigned\n,43.64869,-79.38544


In [6]:

neighbors = neighbors[neighbors.Borough != 'Not assigned\n']


In [7]:
 for i in neighbors['Neighborhood']:
        if i == 'Not assigned\n':
            neighbors['Neighborhood'] = neighbors['Borough']

In [8]:
neighbors = neighbors.replace(r'\n',  ' ', regex=True)

In [9]:
neighbors

,PostalCode,Borough,Neighborhood,Latitude,Longitude
2,M3A,North York,Parkwoods,43.75188,-79.33036
3,M4A,North York,Victoria Village,43.73042,-79.31282
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65514,-79.36265
5,M6A,North York,"Lawrence Manor, Lawrence Heights",43.72321,-79.45141
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.66449,-79.39302
...,...,...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.65369,-79.51112
165,M4Y,Downtown Toronto,Church and Wellesley,43.66659,-79.38130
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.64869,-79.38544
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.63288,-79.48955


In [10]:
downtown_toronto_data = neighbors[neighbors['Borough'] == 'Downtown Toronto ']
# eliminate 'Postalcode' column
downtown_toronto_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65514,-79.36265
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.66449,-79.39302
13,M5B,Downtown Toronto,"Garden District, Ryerson",43.65736,-79.37818
22,M5C,Downtown Toronto,St. James Town,43.65143,-79.37557
31,M5E,Downtown Toronto,Berczy Park,43.64531,-79.37368


In [11]:
CLIENT_ID = 'GYLFCYRMJHYKEGH3AT1K00UZORNDQ3PAHMKRCLV5OLU1DZGV' # your Foursquare ID
CLIENT_SECRET = 'PZWJGRAMCNXIXNALQZCD510PXOQB0MCA1A2WMUCYPKXZIH3K' # your Foursquare Secret
VERSION = '20200602' # Foursquare API version
limit=20
from geopy.geocoders import Nominatim

In [12]:
address = 'Downtown Toronto, ON, Canada'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude_downtown_toronto = location.latitude
longitude_downtown_toronto = location.longitude
print("Downtown Toronto","latitude",latitude_downtown_toronto, "& " "longitude" ,longitude_downtown_toronto)

C:\Users\joaof\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.22.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


Downtown Toronto latitude 43.6563221 & longitude -79.3809161


In [13]:
# create map of Downtown Toronto using latitude and longitude values
map_downtown_toronto = folium.Map(location=[latitude_downtown_toronto,longitude_downtown_toronto], zoom_start=11)

# add markers to map
for lat, lng, label in zip(downtown_toronto_data['Latitude'], downtown_toronto_data['Longitude'], downtown_toronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_downtown_toronto)  
    
map_downtown_toronto

In [14]:
from folium import plugins
# create map of Downtown Toronto using latitude and longitude values
map_downtown_toronto = folium.Map(location=[latitude_downtown_toronto,longitude_downtown_toronto], zoom_start=11)
# instantiate a mark cluster object for the incidents in the dataframe
incidents = plugins.MarkerCluster().add_to(map_downtown_toronto)
# add markers to map
for lat, lng, label in zip(downtown_toronto_data['Latitude'], downtown_toronto_data['Longitude'], downtown_toronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(incidents)  
    
map_downtown_toronto

## End of "Segmenting and Clustering Neighborhoods in Toronto"

# Introduction

This project aims to help markteers and business owners understand the relation between location and the type of venues present on thoose locations, to make it easier to predict where it would be more profitable to start or expand a business in Toronto.

## DATA DESCRIPTION

For this problem, we will get the services of Foursquare API to explore the data of the citie of Toronto, in terms of it's neighborhoods. The data also include the information about the places around each neighborhood like restaurants, hotels, coffee shops, parks, theaters, art galleries, museums and many more. We selected one Borough from each city to analyze their neighborhoods. Downtown Toronto from Toronto. We will use machine learning technique, “Clustering” to segment the neighborhoods with similar objects on the basis of each neighborhood data. These objects will be given priority on the basis of foot traffic (activity) in their respective neighborhoods. This will help to locate the tourist’s areas and hubs.

## METHODOLOGY

We first extracted table of Toronto’s Borough from Wikipedia page, then we arrange the data according to our requirements in the previous module. The arrangement phase we eliminatted “Not assigned” values, combine neighborhoods which have same geographical coordinates at each borough, etc. For data verification and further exploration, we use Foursquare API to get the coordinates of Downtown Toronto and explore its neighborhoods. The neighborhoods are further characterized as venues and venue categories.

In [15]:
# Let's create a function to repeat the process to all the neighborhoods in Toronto
def getNearbyVenues(names, latitudes,longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names,latitudes,longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [34]:
# Write the code to run the above function on each neighborhood and create a new dataframe called toronto_venues.
downtown_toronto_venues = getNearbyVenues(names=downtown_toronto_data['Neighborhood'],
                                   latitudes=downtown_toronto_data['Latitude'],
                                   longitudes=downtown_toronto_data['Longitude'],
                                  )

Regent Park, Harbourfront 
Queen's Park, Ontario Provincial Government 
Garden District, Ryerson 
St. James Town 
Berczy Park 
Central Bay Street 
Christie 
Richmond, Adelaide, King 
Harbourfront East, Union Station, Toronto Islands 
Toronto Dominion Centre, Design Exchange 
Commerce Court, Victoria Hotel 
University of Toronto, Harbord 
Kensington Market, Chinatown, Grange Park 
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport 
Rosedale 
Stn A PO Boxes 
St. James Town, Cabbagetown 
First Canadian Place, Underground city 
Church and Wellesley 


In [35]:
print(downtown_toronto_venues.shape)
downtown_toronto_venues.head()

(356, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65514,-79.36265,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65514,-79.36265,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65514,-79.36265,Figs Breakfast & Lunch,43.655675,-79.364503,Breakfast Spot
3,"Regent Park, Harbourfront",43.65514,-79.36265,The Yoga Lounge,43.655515,-79.364955,Yoga Studio
4,"Regent Park, Harbourfront",43.65514,-79.36265,Body Blitz Spa East,43.654735,-79.359874,Spa


In [36]:
downtown_toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,20,20,20,20,20,20
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",20,20,20,20,20,20
Central Bay Street,20,20,20,20,20,20
Christie,10,10,10,10,10,10
Church and Wellesley,20,20,20,20,20,20
"Commerce Court, Victoria Hotel",20,20,20,20,20,20
"First Canadian Place, Underground city",20,20,20,20,20,20
"Garden District, Ryerson",20,20,20,20,20,20
"Harbourfront East, Union Station, Toronto Islands",20,20,20,20,20,20


In [37]:
print('There are {} uniques categories.'.format(len(downtown_toronto_venues['Venue Category'].unique())))

There are 124 uniques categories.


In [38]:
# one hot encoding
downtown_toronto_onehot = pd.get_dummies(downtown_toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
downtown_toronto_onehot['Neighborhood'] = downtown_toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [downtown_toronto_onehot.columns[-1]] + list(downtown_toronto_onehot.columns[:-1])
downtown_toronto_onehot = downtown_toronto_onehot[fixed_columns]

downtown_toronto_onehot.head()

,Yoga Studio,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,BBQ Joint,Baby Store,Bakery,Bar,Basketball Stadium,...,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Train Station,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop
0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [39]:
downtown_toronto_grouped = downtown_toronto_onehot.groupby('Neighborhood').mean().reset_index()

In [40]:
# Let's print each neighborhood along with the top 5 most common venues
num_top_venues = 5

for hood in downtown_toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = downtown_toronto_grouped[downtown_toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park ----
          venue  freq
0  Cocktail Bar  0.10
1     Jazz Club  0.05
2   Tailor Shop  0.05
3  Concert Hall  0.05
4          Park  0.05


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport ----
                venue  freq
0  Italian Restaurant  0.10
1         Yoga Studio  0.05
2             Brewery  0.05
3           Speakeasy  0.05
4      Sandwich Place  0.05


----Central Bay Street ----
                 venue  freq
0          Coffee Shop  0.15
1  Japanese Restaurant  0.10
2                Hotel  0.05
3     Sushi Restaurant  0.05
4            Gastropub  0.05


----Christie ----
                venue  freq
0                Café   0.3
1       Grocery Store   0.2
2  Italian Restaurant   0.1
3          Playground   0.1
4         Coffee Shop   0.1


----Church and Wellesley ----
        venue  freq
0     Gay Bar  0.10
1   Wine Shop  0.05
2   Bookstore  0.05
3        Park  0.05
4  Restaurant  0.05


----Commerce Co

In [41]:
# Let's put that into a pandas dataframe
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [42]:
# Now let's create the new dataframe and display the top 10 venues for each neighborhood.
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = downtown_toronto_grouped['Neighborhood']

for ind in np.arange(downtown_toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(downtown_toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Cocktail Bar,Breakfast Spot,Tailor Shop,Café,Liquor Store,Seafood Restaurant,Jazz Club,Concert Hall,Beer Bar,Farmers Market
1,"CN Tower, King and Spadina, Railway Lands, Har...",Italian Restaurant,Hotel,Hotel Bar,Ramen Restaurant,Park,Mexican Restaurant,Men's Store,Market,Gym / Fitness Center,Sandwich Place
2,Central Bay Street,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Clothing Store,Comic Shop,Modern European Restaurant,Poke Place,Miscellaneous Shop,Ramen Restaurant,Middle Eastern Restaurant
3,Christie,Café,Grocery Store,Baby Store,Italian Restaurant,Coffee Shop,Candy Store,Playground,BBQ Joint,Farm,Comic Shop
4,Church and Wellesley,Gay Bar,Wine Shop,Steakhouse,Park,Pizza Place,Men's Store,Dance Studio,Café,Restaurant,Burger Joint
5,"Commerce Court, Victoria Hotel",Café,Coffee Shop,Restaurant,Ice Cream Shop,Museum,Pub,Japanese Restaurant,French Restaurant,Tailor Shop,Hotel
6,"First Canadian Place, Underground city",Café,Restaurant,Coffee Shop,Gluten-free Restaurant,Seafood Restaurant,Pub,Bookstore,Japanese Restaurant,Steakhouse,Pizza Place
7,"Garden District, Ryerson",Café,Hotel,Pizza Place,Sandwich Place,Music Venue,Diner,Ramen Restaurant,Japanese Restaurant,Mexican Restaurant,Plaza
8,"Harbourfront East, Union Station, Toronto Isla...",Hotel,Coffee Shop,Lounge,Basketball Stadium,Italian Restaurant,Bistro,Sporting Goods Shop,Plaza,Supermarket,Beer Bar
9,"Kensington Market, Chinatown, Grange Park",Café,Vietnamese Restaurant,Gaming Cafe,Bakery,Dessert Shop,Record Shop,Caribbean Restaurant,Cheese Shop,Farmers Market,Cocktail Bar


In [43]:
# set number of clusters
kclusters = 5

downtown_toronto_grouped_clustering = downtown_toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(downtown_toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:18] 

array([3, 3, 0, 4, 2, 2, 2, 4, 0, 4, 0, 2, 2, 1, 0, 3, 2, 2])

In [44]:
downtown_toronto_grouped.head()

,Neighborhood,Yoga Studio,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,BBQ Joint,Baby Store,Bakery,Bar,...,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Train Station,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop
0,Berczy Park,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.05,0.0,...,0.00,0.0,0.05,0.0,0.00,0.0,0.05,0.0,0.0,0.00
1,"CN Tower, King and Spadina, Railway Lands, Har...",0.05,0.0,0.0,0.00,0.0,0.0,0.0,0.05,0.0,...,0.00,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.00
2,Central Bay Street,0.00,0.0,0.0,0.05,0.0,0.0,0.0,0.00,0.0,...,0.05,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.00
3,Christie,0.00,0.0,0.0,0.00,0.0,0.0,0.1,0.00,0.0,...,0.00,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.00
4,Church and Wellesley,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,...,0.00,0.0,0.05,0.0,0.05,0.0,0.00,0.0,0.0,0.05


In [45]:
# Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
downtown_toronto_merged = neighborhoods_venues_sorted

# add clustering labels
downtown_toronto_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
downtown_toronto_merged =  downtown_toronto_merged.join(downtown_toronto_venues.set_index('Neighborhood'), on='Neighborhood')

downtown_toronto_merged.head() # check the last columns!

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Berczy Park,Cocktail Bar,Breakfast Spot,Tailor Shop,Café,Liquor Store,Seafood Restaurant,Jazz Club,Concert Hall,Beer Bar,Farmers Market,3,43.64531,-79.37368,The Keg Steakhouse + Bar - Esplanade,43.646712,-79.374768,Restaurant
0,Berczy Park,Cocktail Bar,Breakfast Spot,Tailor Shop,Café,Liquor Store,Seafood Restaurant,Jazz Club,Concert Hall,Beer Bar,Farmers Market,3,43.64531,-79.37368,Fresh On Front,43.647815,-79.374453,Vegetarian / Vegan Restaurant
0,Berczy Park,Cocktail Bar,Breakfast Spot,Tailor Shop,Café,Liquor Store,Seafood Restaurant,Jazz Club,Concert Hall,Beer Bar,Farmers Market,3,43.64531,-79.37368,Meridian Hall,43.646292,-79.376022,Concert Hall
0,Berczy Park,Cocktail Bar,Breakfast Spot,Tailor Shop,Café,Liquor Store,Seafood Restaurant,Jazz Club,Concert Hall,Beer Bar,Farmers Market,3,43.64531,-79.37368,LCBO,43.642944,-79.372440,Liquor Store
0,Berczy Park,Cocktail Bar,Breakfast Spot,Tailor Shop,Café,Liquor Store,Seafood Restaurant,Jazz Club,Concert Hall,Beer Bar,Farmers Market,3,43.64531,-79.37368,Hockey Hall Of Fame (Hockey Hall of Fame),43.646974,-79.377323,Museum


# Results

In [46]:
# create map
map_clusters = folium.Map(location=[latitude_downtown_toronto, longitude_downtown_toronto], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(downtown_toronto_merged['Venue Latitude'], downtown_toronto_merged['Venue Longitude'], downtown_toronto_merged['Neighborhood'], downtown_toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Discussion

In [55]:
downtown_toronto_merged.loc[downtown_toronto_merged['Cluster Labels'] == 0, downtown_toronto_merged.columns[[0] + list(range(1, downtown_toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
2,Central Bay Street,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Clothing Store,Comic Shop,Modern European Restaurant,Poke Place,Miscellaneous Shop,Ramen Restaurant,Middle Eastern Restaurant,0,43.65609,-79.38493,Chatime 日出茶太,43.655542,-79.384684,Bubble Tea Shop
2,Central Bay Street,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Clothing Store,Comic Shop,Modern European Restaurant,Poke Place,Miscellaneous Shop,Ramen Restaurant,Middle Eastern Restaurant,0,43.65609,-79.38493,The Elm Tree Restaurant,43.657397,-79.383761,Modern European Restaurant
2,Central Bay Street,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Clothing Store,Comic Shop,Modern European Restaurant,Poke Place,Miscellaneous Shop,Ramen Restaurant,Middle Eastern Restaurant,0,43.65609,-79.38493,MUJI,43.656024,-79.383284,Miscellaneous Shop
2,Central Bay Street,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Clothing Store,Comic Shop,Modern European Restaurant,Poke Place,Miscellaneous Shop,Ramen Restaurant,Middle Eastern Restaurant,0,43.65609,-79.38493,Sansotei Ramen 三草亭,43.655157,-79.386501,Ramen Restaurant
2,Central Bay Street,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Clothing Store,Comic Shop,Modern European Restaurant,Poke Place,Miscellaneous Shop,Ramen Restaurant,Middle Eastern Restaurant,0,43.65609,-79.38493,The Queen and Beaver Public House,43.657472,-79.383524,Gastropub
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14,St. James Town,Gastropub,Coffee Shop,Japanese Restaurant,Hotel,Italian Restaurant,Restaurant,Creperie,Cosmetics Shop,Food Truck,Poke Place,0,43.65143,-79.37557,Beerbistro,43.649419,-79.377237,Gastropub
14,St. James Town,Gastropub,Coffee Shop,Japanese Restaurant,Hotel,Italian Restaurant,Restaurant,Creperie,Cosmetics Shop,Food Truck,Poke Place,0,43.65143,-79.37557,Richmond Station,43.651569,-79.379266,American Restaurant
14,St. James Town,Gastropub,Coffee Shop,Japanese Restaurant,Hotel,Italian Restaurant,Restaurant,Creperie,Cosmetics Shop,Food Truck,Poke Place,0,43.65143,-79.37557,Triple A Bar (AAA),43.651658,-79.372720,BBQ Joint
14,St. James Town,Gastropub,Coffee Shop,Japanese Restaurant,Hotel,Italian Restaurant,Restaurant,Creperie,Cosmetics Shop,Food Truck,Poke Place,0,43.65143,-79.37557,Elgin And Winter Garden Theatres,43.653394,-79.378507,Theater


In [56]:
downtown_toronto_merged.loc[downtown_toronto_merged['Cluster Labels'] == 1, downtown_toronto_merged.columns[[0] + list(range(1, downtown_toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
13,Rosedale,Playground,Tennis Court,Campground,Shop & Service,Bike Trail,Park,Gastropub,Dessert Shop,College Gym,College Rec Center,1,43.6819,-79.3785,Rosedale Tennis Club,43.683226,-79.378984,Tennis Court
13,Rosedale,Playground,Tennis Court,Campground,Shop & Service,Bike Trail,Park,Gastropub,Dessert Shop,College Gym,College Rec Center,1,43.6819,-79.3785,Rosedale Park,43.682328,-79.378934,Playground
13,Rosedale,Playground,Tennis Court,Campground,Shop & Service,Bike Trail,Park,Gastropub,Dessert Shop,College Gym,College Rec Center,1,43.6819,-79.3785,Betline Trail at Roxborough dr.,43.680530,-79.381490,Bike Trail
13,Rosedale,Playground,Tennis Court,Campground,Shop & Service,Bike Trail,Park,Gastropub,Dessert Shop,College Gym,College Rec Center,1,43.6819,-79.3785,Whitney Park,43.682036,-79.373788,Park
13,Rosedale,Playground,Tennis Court,Campground,Shop & Service,Bike Trail,Park,Gastropub,Dessert Shop,College Gym,College Rec Center,1,43.6819,-79.3785,Mooredale Day Camp,43.678332,-79.380491,Campground
13,Rosedale,Playground,Tennis Court,Campground,Shop & Service,Bike Trail,Park,Gastropub,Dessert Shop,College Gym,College Rec Center,1,43.6819,-79.3785,Petite & Sweet,43.685982,-79.376072,Shop & Service


In [58]:
downtown_toronto_merged.loc[downtown_toronto_merged['Cluster Labels'] == 2, downtown_toronto_merged.columns[[0] + list(range(1, downtown_toronto_merged.shape[1]))]]


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
4,Church and Wellesley,Gay Bar,Wine Shop,Steakhouse,Park,Pizza Place,Men's Store,Dance Studio,Café,Restaurant,Burger Joint,2,43.66659,-79.38130,Storm Crow Manor,43.666840,-79.381593,Theme Restaurant
4,Church and Wellesley,Gay Bar,Wine Shop,Steakhouse,Park,Pizza Place,Men's Store,Dance Studio,Café,Restaurant,Burger Joint,2,43.66659,-79.38130,Smith,43.666927,-79.381421,Breakfast Spot
4,Church and Wellesley,Gay Bar,Wine Shop,Steakhouse,Park,Pizza Place,Men's Store,Dance Studio,Café,Restaurant,Burger Joint,2,43.66659,-79.38130,The Keg Steakhouse + Bar,43.666756,-79.378302,Steakhouse
4,Church and Wellesley,Gay Bar,Wine Shop,Steakhouse,Park,Pizza Place,Men's Store,Dance Studio,Café,Restaurant,Burger Joint,2,43.66659,-79.38130,Fabarnak,43.666377,-79.380964,Restaurant
4,Church and Wellesley,Gay Bar,Wine Shop,Steakhouse,Park,Pizza Place,Men's Store,Dance Studio,Café,Restaurant,Burger Joint,2,43.66659,-79.38130,Glad Day Bookshop,43.665271,-79.380785,Bookstore
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17,"Toronto Dominion Centre, Design Exchange",Coffee Shop,Café,Beer Bar,Museum,Pizza Place,Pub,Restaurant,Deli / Bodega,Sandwich Place,Japanese Restaurant,2,43.64710,-79.38153,Union Pearson Express,43.644362,-79.383199,Train Station
17,"Toronto Dominion Centre, Design Exchange",Coffee Shop,Café,Beer Bar,Museum,Pizza Place,Pub,Restaurant,Deli / Bodega,Sandwich Place,Japanese Restaurant,2,43.64710,-79.38153,Hockey Hall Of Fame (Hockey Hall of Fame),43.646974,-79.377323,Museum
17,"Toronto Dominion Centre, Design Exchange",Coffee Shop,Café,Beer Bar,Museum,Pizza Place,Pub,Restaurant,Deli / Bodega,Sandwich Place,Japanese Restaurant,2,43.64710,-79.38153,Pizzeria Libretto,43.648334,-79.385111,Pizza Place
17,"Toronto Dominion Centre, Design Exchange",Coffee Shop,Café,Beer Bar,Museum,Pizza Place,Pub,Restaurant,Deli / Bodega,Sandwich Place,Japanese Restaurant,2,43.64710,-79.38153,Petit Four Bakery,43.647744,-79.379588,Sandwich Place


In [59]:
downtown_toronto_merged.loc[downtown_toronto_merged['Cluster Labels'] == 3, downtown_toronto_merged.columns[[0] + list(range(1, downtown_toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Berczy Park,Cocktail Bar,Breakfast Spot,Tailor Shop,Café,Liquor Store,Seafood Restaurant,Jazz Club,Concert Hall,Beer Bar,Farmers Market,3,43.64531,-79.37368,The Keg Steakhouse + Bar - Esplanade,43.646712,-79.374768,Restaurant
0,Berczy Park,Cocktail Bar,Breakfast Spot,Tailor Shop,Café,Liquor Store,Seafood Restaurant,Jazz Club,Concert Hall,Beer Bar,Farmers Market,3,43.64531,-79.37368,Fresh On Front,43.647815,-79.374453,Vegetarian / Vegan Restaurant
0,Berczy Park,Cocktail Bar,Breakfast Spot,Tailor Shop,Café,Liquor Store,Seafood Restaurant,Jazz Club,Concert Hall,Beer Bar,Farmers Market,3,43.64531,-79.37368,Meridian Hall,43.646292,-79.376022,Concert Hall
0,Berczy Park,Cocktail Bar,Breakfast Spot,Tailor Shop,Café,Liquor Store,Seafood Restaurant,Jazz Club,Concert Hall,Beer Bar,Farmers Market,3,43.64531,-79.37368,LCBO,43.642944,-79.372440,Liquor Store
0,Berczy Park,Cocktail Bar,Breakfast Spot,Tailor Shop,Café,Liquor Store,Seafood Restaurant,Jazz Club,Concert Hall,Beer Bar,Farmers Market,3,43.64531,-79.37368,Hockey Hall Of Fame (Hockey Hall of Fame),43.646974,-79.377323,Museum
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18,"University of Toronto, Harbord",Restaurant,Bookstore,Bakery,Yoga Studio,Sushi Restaurant,Comfort Food Restaurant,Greek Restaurant,Dessert Shop,Café,Japanese Restaurant,3,43.66311,-79.40180,Athletic Centre,43.662487,-79.400657,College Gym
18,"University of Toronto, Harbord",Restaurant,Bookstore,Bakery,Yoga Studio,Sushi Restaurant,Comfort Food Restaurant,Greek Restaurant,Dessert Shop,Café,Japanese Restaurant,3,43.66311,-79.40180,Real Thailand Restaurant,43.666557,-79.404657,Thai Restaurant
18,"University of Toronto, Harbord",Restaurant,Bookstore,Bakery,Yoga Studio,Sushi Restaurant,Comfort Food Restaurant,Greek Restaurant,Dessert Shop,Café,Japanese Restaurant,3,43.66311,-79.40180,Innis Cafe,43.665401,-79.399715,Café
18,"University of Toronto, Harbord",Restaurant,Bookstore,Bakery,Yoga Studio,Sushi Restaurant,Comfort Food Restaurant,Greek Restaurant,Dessert Shop,Café,Japanese Restaurant,3,43.66311,-79.40180,The Original Gyro Grill,43.666621,-79.405544,Greek Restaurant


In [54]:
downtown_toronto_merged.loc[downtown_toronto_merged['Cluster Labels'] == 4, downtown_toronto_merged.columns[[0] + list(range(1, downtown_toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
3,Christie,Café,Grocery Store,Baby Store,Italian Restaurant,Coffee Shop,Candy Store,Playground,BBQ Joint,Farm,Comic Shop,4,43.66878,-79.42071,Fiesta Farms,43.668471,-79.420485,Grocery Store
3,Christie,Café,Grocery Store,Baby Store,Italian Restaurant,Coffee Shop,Candy Store,Playground,BBQ Joint,Farm,Comic Shop,4,43.66878,-79.42071,Scout and Cash Caffe,43.667360,-79.419938,Café
3,Christie,Café,Grocery Store,Baby Store,Italian Restaurant,Coffee Shop,Candy Store,Playground,BBQ Joint,Farm,Comic Shop,4,43.66878,-79.42071,Contra Cafe,43.669107,-79.426105,Café
3,Christie,Café,Grocery Store,Baby Store,Italian Restaurant,Coffee Shop,Candy Store,Playground,BBQ Joint,Farm,Comic Shop,4,43.66878,-79.42071,Starbucks,43.671530,-79.421400,Coffee Shop
3,Christie,Café,Grocery Store,Baby Store,Italian Restaurant,Coffee Shop,Candy Store,Playground,BBQ Joint,Farm,Comic Shop,4,43.66878,-79.42071,Vermont Square Park,43.670493,-79.415399,Playground
3,Christie,Café,Grocery Store,Baby Store,Italian Restaurant,Coffee Shop,Candy Store,Playground,BBQ Joint,Farm,Comic Shop,4,43.66878,-79.42071,Vinny’s Panini,43.670679,-79.426148,Italian Restaurant
3,Christie,Café,Grocery Store,Baby Store,Italian Restaurant,Coffee Shop,Candy Store,Playground,BBQ Joint,Farm,Comic Shop,4,43.66878,-79.42071,Faema Caffe,43.671046,-79.419297,Café
3,Christie,Café,Grocery Store,Baby Store,Italian Restaurant,Coffee Shop,Candy Store,Playground,BBQ Joint,Farm,Comic Shop,4,43.66878,-79.42071,Stubbe Chocolates,43.671566,-79.421289,Candy Store
3,Christie,Café,Grocery Store,Baby Store,Italian Restaurant,Coffee Shop,Candy Store,Playground,BBQ Joint,Farm,Comic Shop,4,43.66878,-79.42071,Loblaws,43.671657,-79.421364,Grocery Store
3,Christie,Café,Grocery Store,Baby Store,Italian Restaurant,Coffee Shop,Candy Store,Playground,BBQ Joint,Farm,Comic Shop,4,43.66878,-79.42071,Marlenes Just Babies,43.671824,-79.420499,Baby Store


## Results

Cluster 5- More oriented for families and residential area. <br>
Cluster 4- University bars, more oriented for young people with many liquor store, jazz clubs,etc. <br>
Cluster 3- Alternative bars and museums, a more cultural part of town. <br>
Cluster 2- Zones of sports practice.<br>
Cluster 1- Restauration.

## Observations

We can observate that the zone of sports practice are very concentrated in one area, while the others clusters are ditrubuted mainly in the center of Downtown Toronto. <br>
Also we can conclude that the venues closer to the sea are usually restaurants and bars.


# Conclusion

There are many criteria in determinate where is the best place to start a business. You may want open your shop nearby stores alike, where you know your main target clients will usually be, or in a place where lacks your type of business so you don't have many competition. <br>
If you going to open a bar you have to think what kind of bar you want and where it makes sense to open one. <br>
This project doesn't aim to determinate mandatorily where you must open your store, but to provide information of where is more common to have a specified type of store and helping you choose the right place depending on your business plans.
